##Cleaning The DataFrames

In [ ]:
#Clean the df_pin Data Frame
from pyspark.sql.functions import expr

# a) Replace empty entries and entries with no relevant data in each column with Nones
df_pin_cleaned = df_pin.na.replace('', None)

# b) Perform the necessary transformations on the follower_count column
df_pin_cleaned = df_pin_cleaned.withColumn("follower_count", df_pin_cleaned["follower_count"].cast("int"))

# c) Ensure that each column containing numeric data has a numeric data type
numeric_columns = ["downloaded", "index"]
for col in numeric_columns:
    df_pin_cleaned = df_pin_cleaned.withColumn(col, df_pin_cleaned[col].cast("double"))

# d) Clean the data in the save_location column
df_pin_cleaned = df_pin_cleaned.withColumn("save_location", expr("substring_index(save_location, '/', -1)"))

# e) Rename the index column to ind
df_pin_cleaned = df_pin_cleaned.withColumnRenamed("index", "ind")

# f) Reorder the DataFrame columns
column_order = ["ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list",
                "is_image_or_video", "image_src", "save_location", "category"]

df_pin_cleaned = df_pin_cleaned.select(column_order)

# Save the cleaned DataFrame to the specified mount point
output_path_pin = "/mnt/diane_mount/cleaned_data/df_pin_cleaned"
df_pin_cleaned.write.mode("overwrite").parquet(output_path_pin)

# Show the cleaned DataFrame
df_pin_cleaned.show()


+-------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------+--------------+
 ind| unique_id| title| description|follower_count| poster_name| tag_list|is_image_or_video| image_src| save_location| category|
+-------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------+--------------+
 2482.0|08604f20-fa17-4b9...|FORNT PORCH CHRIS...|Christmas decorat...| null|Life on Summerhil...|Diy Christmas Dec...| video|https://i.pinimg....| christmas| christmas|
10138.0|927c4658-cc3f-4b9...|14 Amazing Things...|This Costa Rica i...| null|Wanderlust Chloe ...|Costa Rica Travel...| image|https://i.pinimg....| travel| travel|
 3156.0|fa6e31a4-18c2-4ec...|Handprint Reindee...|This post may con...| null|Michelle {CraftyM...|Christmas Gifts F...| image|https://i.pinimg....|diy-and-crafts|diy-and-crafts|
 3419.0|d0b80187-0171-49b...|Easy Christmas Tr...|Easy Christmas tr...| null|Kids Crafts & Fre...|Christmas Crafts ...| image|https://i.pinimg....|diy-and-crafts|diy-and-crafts|
 5494.0|8fb2af68-543b-463...|Dave Ramsey's 7 B...|If you love budge...| null|Living Low Key | ...|Financial Peace,F...| image|https://i.pinimg....| finance| finance|
 2418.0|da8745a6-5160-46c...|100 DIY Christmas...|Here are the best...| null| HikenDip|Farmhouse Christm...| image|https://i.pinimg....| christmas| christmas|
 7922.0|a584581c-1b38-473...|45 Top Life Quote...|summcoco gives yo...| null|Sumcoco | Decor I...|Life Quotes Love,...| image|https://i.pinimg....| quotes| quotes|
 771.0|a5021766-a8aa-4dc...|African Sunset Sh...|Use your mini wor...| null|Taming Little Mon...|African Art Proje...| image|https://i.pinimg....| art| art|
 4508.0|9064f4a2-2753-476...|Custom Event and ...|Personalize your ...| 111|Life of Neon | Cu...|Our Wedding,Weddi...| image|https://i.pinimg....|event-planning|event-planning|
 4076.0|3a52d364-7c04-47c...|Phonics Activitie...|Hi everyone! As a...| null| The Literacy Nest|Literacy Games,Ki...| image|https://i.pinimg....| education| education|
 2293.0|1fd7d4cc-54c1-454...|Over 40 of the BE...|Over 40 of the BE...| null|Kitchen Fun With ...|Diy Christmas Lig...| image|https://i.pinimg....| christmas| christmas|
 159.0|841a161a-47b8-416...|Valentine's Day B...|This bee directed...| null|Teachers Pay Teac...|Classroom Art Pro...| image|https://i.pinimg....| art| art|
 9979.0|2b2abc85-fc51-481...|Paris in the Summ...|Are you traveling...| null| Petite in Paris|Torre Eiffel Pari...| image|https://i.pinimg....| travel| travel|
 9759.0|d105eb6e-0f9f-46e...|8 Best Greek Isla...|Although you'd th...| null|TheFab20s | Trave...|Greek Islands To ...| image|https://i.pinimg....| travel| travel|
 2698.0|73f16302-4871-486...|DIY Dollar Tree V...|DIY Dollar Store ...| null| South Lumina Style|Valentine Day Wre...| image|https://i.pinimg....|diy-and-crafts|diy-and-crafts|
 6145.0|82e13a07-db99-43a...|HOLIDAY MANTLE DE...|Holiday mantle de...| null| Stylin by Aylin|Winter Home Decor...| image|https://i.pinimg....| home-decor| home-decor|
 427.0|4a455340-09a2-437...|DIY Acrylic Paint...|Easy to follow st...| null| Abbotts At Home|Acrylic Pouring A...| video|https://i.pinimg....| art| art|
 1704.0|5fbf9863-fb79-477...|Christmas Trees F...|Christmas Trees F...| null| Instructables|Pallet Wood Chris...| image|https://i.pinimg....| christmas| christmas|
 4315.0|21b59ba9-829d-4c3...|Podcasts for Teac...|Podcasts for Teac...| null| Math Giraffe|Middle School Cla...| image|https://i.pinimg....| education| education|
 4315.0|21b59ba9-829d-4c3...|Podcasts for Teac...|Podcasts for Teac...| null| Math Giraffe|Middle School Cla...| image|https://i.pinimg....| education| education|
+-------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+------

In [ ]:
#Clean the geo data

from pyspark.sql.functions import col, array, to_timestamp, expr

# a) Create a new column 'coordinates' that contains an array based on the latitude and longitude columns
df_geo_cleaned = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))

# b) Drop the latitude and longitude columns from the DataFrame
df_geo_cleaned = df_geo_cleaned.drop("latitude", "longitude")

# c) Convert the timestamp column from a string to a timestamp data type
df_geo_cleaned = df_geo_cleaned.withColumn("timestamp", to_timestamp(col("timestamp")))

# d) Reorder the DataFrame columns
column_order_geo = ["ind", "country", "coordinates", "timestamp"]
df_geo_cleaned = df_geo_cleaned.select(column_order_geo)

# Save the cleaned DataFrame to the specified mount point
output_path_geo = "/mnt/diane_mount/cleaned_data/df_geo_cleaned"
df_geo_cleaned.write.mode("overwrite").parquet(output_path_geo)

# Show the cleaned DataFrame
df_geo_cleaned.show()


+-----+--------------------+--------------------+-------------------+
 ind| country| coordinates| timestamp|
+-----+--------------------+--------------------+-------------------+
 2418|Antarctica (the t...|[-88.4642, -171.061]|2022-05-27 11:30:59|
10794|Cocos (Keeling) I...|[-89.5236, -154.567]|2022-01-01 02:26:50|
 2074|Central African R...| [-52.3213, -50.11]|2019-11-03 05:41:59|
 2293|British Virgin Is...|[-87.7946, -159.647]|2022-03-21 10:46:53|
10663|Saint Kitts and N...| [-27.3474, -162.83]|2019-07-25 18:53:51|
 7922| Antigua and Barbuda|[-88.0974, -172.052]|2021-01-27 09:14:19|
 8606| Antigua and Barbuda|[-88.0974, -172.052]|2021-03-28 14:54:07|
 603|Netherlands Antilles| [14.0083, -141.603]|2019-06-25 05:13:01|
 9979| Dominican Republic| [14.9967, -120.682]|2018-07-18 19:01:46|
 1704| Christmas Island|[-56.8702, -93.6232]|2018-11-29 04:46:39|
 7790| Papua New Guinea| [-43.692, 64.9839]|2018-07-31 08:19:15|
 9759| American Samoa|[-77.9744, -106.258]|2017-12-30 13:05:49|
 1422| American Samoa|[-88.5252, -172.436]|2018-04-30 08:27:21|
 2923| Cote d'Ivoire|[-84.6302, -164.507]|2019-09-08 22:53:09|
 8304| French Guiana| [-28.8852, -164.87]|2019-09-13 04:50:29|
 4315| Cote d'Ivoire| [-45.8508, 66.1003]|2019-12-15 03:51:28|
 8578| Guinea-Bissau| [34.6427, -161.397]|2018-01-26 19:27:11|
 6844| New Caledonia| [-22.6915, 5.69245]|2021-06-06 21:53:11|
 3599| Afghanistan|[-88.5478, -174.971]|2019-03-03 06:13:41|
 5069| Azerbaijan|[-63.0063, -157.474]|2021-03-20 09:32:44|
+-----+--------------------+--------------------+-------------------+
only showing top 20 rows

In [ ]:
#clean the user data frame

from pyspark.sql.functions import col, concat_ws, to_timestamp

# a) Create a new column 'user_name' that concatenates the information found in the first_name and last_name columns
df_user_cleaned = df_user.withColumn("user_name", concat_ws(" ", col("first_name"), col("last_name")))

# b) Drop the first_name and last_name columns from the DataFrame
df_user_cleaned = df_user_cleaned.drop("first_name", "last_name")

# c) Convert the date_joined column from a string to a timestamp data type
df_user_cleaned = df_user_cleaned.withColumn("date_joined", to_timestamp(col("date_joined")))

# d) Reorder the DataFrame columns
column_order_user = ["ind", "user_name", "age", "date_joined"]
df_user_cleaned = df_user_cleaned.select(column_order_user)

# Save the cleaned DataFrame to the specified mount point
output_path_user = "/mnt/diane_mount/cleaned_data/df_user_cleaned"
df_user_cleaned.write.mode("overwrite").parquet(output_path_user)

# Show the cleaned DataFrame
df_user_cleaned.show()


+-----+--------------------+---+-------------------+
 ind| user_name|age| date_joined|
+-----+--------------------+---+-------------------+
 2015|Christopher Bradshaw| 27|2016-03-08 13:38:37|
 6398| Christina Davenport| 39|2016-06-29 20:43:59|
 3599| Alexandria Alvarado| 20|2015-10-23 04:13:23|
 5076| Christopher Butler| 20|2015-12-01 15:08:31|
 7790| Michelle Gutierrez| 39|2017-07-19 07:12:04|
10509| Brittany Thompson| 49|2016-04-22 20:36:02|
 8731| Andrea Alexander| 21|2015-11-10 09:27:42|
 8887| Austin Rodriguez| 24|2016-03-31 20:56:39|
 7768| Christine Cortez| 23|2015-12-01 18:15:02|
 4315| Michelle Prince| 36|2015-12-20 16:38:13|
10625| Christian Lang| 32|2017-10-10 20:09:33|
 9672| Jennifer Hudson| 22|2016-02-11 20:46:04|
 8930| Andrew Anderson| 23|2015-11-28 11:52:37|
 6566| Alexander Perez| 31|2017-08-04 14:30:22|
 3729| Richard Edwards| 52|2016-02-07 20:00:25|
10552| Michael Hunter| 40|2017-05-16 07:09:21|
 1313| Brittany Jones| 32|2016-04-02 03:51:23|
10794| Thomas Turner| 34|2016-12-22 00:02:02|
 2959| David Griffith| 20|2016-01-07 19:49:22|
 9875| Brendan Joseph| 26|2015-12-20 10:28:00|
+-----+--------------------+---+-------------------+
only showing top 20 rows

##Checking the Data Frames

In [ ]:
display(df_geo)

country,ind,latitude,longitude,timestamp
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27 11:30:59
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01 02:26:50
Central African Republic,2074,-52.3213,-50.11,2019-11-03 05:41:59
British Virgin Islands,2293,-87.7946,-159.647,2022-03-21 10:46:53
Saint Kitts and Nevis,10663,-27.3474,-162.83,2019-07-25 18:53:51
Antigua and Barbuda,7922,-88.0974,-172.052,2021-01-27 09:14:19
Antigua and Barbuda,8606,-88.0974,-172.052,2021-03-28 14:54:07
Netherlands Antilles,603,14.0083,-141.603,2019-06-25 05:13:01
Dominican Republic,9979,14.9967,-120.682,2018-07-18 19:01:46
Christmas Island,1704,-56.8702,-93.6232,2018-11-29 04:46:39


In [ ]:
display(df_pin)

category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
christmas,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",1,46k,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,2482,video,"Life on Summerhill | Home, Holiday Decor & DIY Website",Local save in /data/christmas,"Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",FORNT PORCH CHRISTMAS DECORATING IDEAS,08604f20-fa17-4b9a-9949-781717eca6cd
travel,"This Costa Rica itinerary is the ultimate guide to spending two weeks in Costa Rica. Find out about visiting La Fortuna, Arenal, Monteverde, Naranjo, Corcovado National Park, Or…",1,10k,https://i.pinimg.com/originals/30/93/cb/3093cb01d9de2d125fda8ba5e3e41946.jpg,10138,image,"Wanderlust Chloe ✈️ Travel guides, inspo and adventure travel ✈️",Local save in /data/travel,"Costa Rica Travel,Rio Celeste Costa Rica,Dream Vacations,Vacation Spots,Vacation Travel,Travel Pictures,Travel Photos,Fortuna Costa Rica,Costa Rica Pictures","14 Amazing Things To Do In Costa Rica | Volcanoes, Waterfalls, Wildlife And More",927c4658-cc3f-4b92-9b5c-70743d0c238d
diy-and-crafts,"This post may contain affiliate links, read our Disclosure Policy for more information. As an Amazon Associate I earn from qualifying purchases, thank you! Make some cute handpr…",1,892k,https://i.pinimg.com/originals/ff/fe/38/fffe384f3ec18a0d87cb2d80cc8c1499.jpg,3156,image,Michelle {CraftyMorning.com},Local save in /data/diy-and-crafts,"Christmas Gifts For Parents,Christmas Decorations For Kids,Christmas Crafts For Toddlers,Preschool Christmas,Christmas Crafts For Gifts,Christmas Activities,Toddler Crafts,Kids Christmas,Christmas Feeling",Handprint Reindeer Ornaments - Crafty Morning,fa6e31a4-18c2-4eca-a6d8-e903eee2c2a4
diy-and-crafts,Easy Christmas tree Craft Ideas for toddlers and preschoolers. Engage your kids in these DIY,1,3k,https://i.pinimg.com/originals/69/f0/75/69f075939d4449dffa69519756c30e26.png,3419,image,Kids Crafts & Free Preschool Printables- Sharing Our Experiences,Local save in /data/diy-and-crafts,"Christmas Crafts For Kids To Make,Christmas Tree Painting,Christmas Activities For Kids,Easy Christmas Crafts For Toddlers,Kid Activities,Christmas Handprint Crafts,Christmas Tree Crafts,Christmas Baby,Xmas Tree",Easy Christmas Tree Crafts Ideas for toddlers and preschoolers | Sharing Our Experiences,d0b80187-0171-49b2-8ee4-572984244f65
finance,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",1,26k,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,5494,image,"Living Low Key | Save Money, Make Money, & Frugal Living",Local save in /data/finance,"Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,8fb2af68-543b-4639-8119-de33d28706ed
christmas,Here are the best DIY Christmas Centerpieces ideas perfect for your Christmas & holiday season home decor. From Christmas Vignettes to Table Centerpieces.,1,500k,https://i.pinimg.com/originals/aa/6d/0f/aa6d0f44d7c1c96b998cb9aa6c4446b8.png,2418,image,HikenDip,Local save in /data/christmas,"Farmhouse Christmas Decor,Rustic Christmas,Christmas Time,Vintage Christmas,Xmas,Primitive Christmas Crafts,Christmas Vignette,Indoor Christmas Decorations,Diy Christmas Ornaments",100 DIY Christmas Centerpieces You'll Love To Decorate Your Home With For The Christmas Season - Hike

In [ ]:
display(df_user)

age,date_joined,first_name,ind,last_name
27,2016-03-08 13:38:37,Christopher,2015,Bradshaw
39,2016-06-29 20:43:59,Christina,6398,Davenport
20,2015-10-23 04:13:23,Alexandria,3599,Alvarado
20,2015-12-01 15:08:31,Christopher,5076,Butler
39,2017-07-19 07:12:04,Michelle,7790,Gutierrez
49,2016-04-22 20:36:02,Brittany,10509,Thompson
21,2015-11-10 09:27:42,Andrea,8731,Alexander
24,2016-03-31 20:56:39,Austin,8887,Rodriguez
23,2015-12-01 18:15:02,Christine,7768,Cortez
36,2015-12-20 16:38:13,Michelle,4315,Prince



##Updated Data Cleaning Section 

In [ ]:
# Datacleaning the df_pin dataframe

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, array, to_timestamp, concat_ws
from pyspark.sql.types import IntegerType

# Create a Spark session
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

# Replace empty entries and entries with no relevant data in each column with Nones
df_pin_cleaned = df_pin.na.replace('', None)

# Perform the necessary transformations on the follower_count column
df_pin_cleaned = df_pin_cleaned.withColumn("follower_count", df_pin_cleaned["follower_count"].cast(IntegerType()))

# Ensure that each column containing numeric data has a numeric data type
numeric_columns = ["downloaded", "index"]
for col_name in numeric_columns:
    df_pin_cleaned = df_pin_cleaned.withColumn(col_name, df_pin_cleaned[col_name].cast("double"))

# Clean the data in the save_location column
df_pin_cleaned = df_pin_cleaned.withColumn("save_location", expr("substring_index(save_location, '/', -1)"))

# Rename the index column to ind
df_pin_cleaned = df_pin_cleaned.withColumnRenamed("index", "ind")

# Reorder the DataFrame columns
column_order_pin = ["ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list",
                    "is_image_or_video", "image_src", "save_location", "category"]

df_pin_cleaned = df_pin_cleaned.select(column_order_pin)

# Get the current timestamp
timestamp_str_pin = spark.sql("SELECT current_timestamp() AS timestamp").collect()[0].timestamp

# Create a new output path with a timestamp
output_path_pin = f"/mnt/diane_mount/cleaned_data/df_pin_cleaned_{timestamp_str_pin}"

# Save the cleaned DataFrame to the new output path
df_pin_cleaned.write.mode("overwrite").parquet(output_path_pin)

# Show the cleaned DataFrame
df_pin_cleaned.show()


+-------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------+--------------+
 ind| unique_id| title| description|follower_count| poster_name| tag_list|is_image_or_video| image_src| save_location| category|
+-------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------+--------------+
 2482.0|08604f20-fa17-4b9...|FORNT PORCH CHRIS...|Christmas decorat...| null|Life on Summerhil...|Diy Christmas Dec...| video|https://i.pinimg....| christmas| christmas|
10138.0|927c4658-cc3f-4b9...|14 Amazing Things...|This Costa Rica i...| null|Wanderlust Chloe ...|Costa Rica Travel...| image|https://i.pinimg....| travel| travel|
 3156.0|fa6e31a4-18c2-4ec...|Handprint Reindee...|This post may con...| null|Michelle {CraftyM...|Christmas Gifts F...| image|https://i.pinimg....|diy-and-crafts|diy-and-crafts|
 3419.0|d0b80187-0171-49b...|Easy Christmas Tr...|Easy Christmas tr...| null|Kids Crafts & Fre...|Christmas Crafts ...| image|https://i.pinimg....|diy-and-crafts|diy-and-crafts|
 5494.0|8fb2af68-543b-463...|Dave Ramsey's 7 B...|If you love budge...| null|Living Low Key | ...|Financial Peace,F...| image|https://i.pinimg....| finance| finance|
 2418.0|da8745a6-5160-46c...|100 DIY Christmas...|Here are the best...| null| HikenDip|Farmhouse Christm...| image|https://i.pinimg....| christmas| christmas|
 7922.0|a584581c-1b38-473...|45 Top Life Quote...|summcoco gives yo...| null|Sumcoco | Decor I...|Life Quotes Love,...| image|https://i.pinimg....| quotes| quotes|
 771.0|a5021766-a8aa-4dc...|African Sunset Sh...|Use your mini wor...| null|Taming Little Mon...|African Art Proje...| image|https://i.pinimg....| art| art|
 4508.0|9064f4a2-2753-476...|Custom Event and ...|Personalize your ...| 111|Life of Neon | Cu...|Our Wedding,Weddi...| image|https://i.pinimg....|event-planning|event-planning|
 4076.0|3a52d364-7c04-47c...|Phonics Activitie...|Hi everyone! As a...| null| The Literacy Nest|Literacy Games,Ki...| image|https://i.pinimg....| education| education|
 2293.0|1fd7d4cc-54c1-454...|Over 40 of the BE...|Over 40 of the BE...| null|Kitchen Fun With ...|Diy Christmas Lig...| image|https://i.pinimg....| christmas| christmas|
 159.0|841a161a-47b8-416...|Valentine's Day B...|This bee directed...| null|Teachers Pay Teac...|Classroom Art Pro...| image|https://i.pinimg....| art| art|
 9979.0|2b2abc85-fc51-481...|Paris in the Summ...|Are you traveling...| null| Petite in Paris|Torre Eiffel Pari...| image|https://i.pinimg....| travel| travel|
 9759.0|d105eb6e-0f9f-46e...|8 Best Greek Isla...|Although you'd th...| null|TheFab20s | Trave...|Greek Islands To ...| image|https://i.pinimg....| travel| travel|
 2698.0|73f16302-4871-486...|DIY Dollar Tree V...|DIY Dollar Store ...| null| South Lumina Style|Valentine Day Wre...| image|https://i.pinimg....|diy-and-crafts|diy-and-crafts|
 6145.0|82e13a07-db99-43a...|HOLIDAY MANTLE DE...|Holiday mantle de...| null| Stylin by Aylin|Winter Home Decor...| image|https://i.pinimg....| home-decor| home-decor|
 427.0|4a455340-09a2-437...|DIY Acrylic Paint...|Easy to follow st...| null| Abbotts At Home|Acrylic Pouring A...| video|https://i.pinimg....| art| art|
 1704.0|5fbf9863-fb79-477...|Christmas Trees F...|Christmas Trees F...| null| Instructables|Pallet Wood Chris...| image|https://i.pinimg....| christmas| christmas|
 4315.0|21b59ba9-829d-4c3...|Podcasts for Teac...|Podcasts for Teac...| null| Math Giraffe|Middle School Cla...| image|https://i.pinimg....| education| education|
 4315.0|21b59ba9-829d-4c3...|Podcasts for Teac...|Podcasts for Teac...| null| Math Giraffe|Middle School Cla...| image|https://i.pinimg....| education| education|
+-------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+------

In [ ]:
#Cleaning the df_geo dataframe

from pyspark.sql.functions import col, array, to_timestamp, expr

# Create a new column 'coordinates' that contains an array based on the latitude and longitude columns
df_geo_cleaned = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))

# Drop the latitude and longitude columns from the DataFrame
df_geo_cleaned = df_geo_cleaned.drop("latitude", "longitude")

# Convert the timestamp column from a string to a timestamp data type
df_geo_cleaned = df_geo_cleaned.withColumn("timestamp", to_timestamp(col("timestamp")))

# Reorder the DataFrame columns
column_order_geo = ["ind", "country", "coordinates", "timestamp"]
df_geo_cleaned = df_geo_cleaned.select(column_order_geo)

# Get the current timestamp
timestamp_str_geo = spark.sql("SELECT current_timestamp() AS timestamp").collect()[0].timestamp

# Create a new output path with a timestamp
output_path_geo = f"/mnt/diane_mount/cleaned_data/df_geo_cleaned_{timestamp_str_geo}"

# Save the cleaned DataFrame to the new output path
df_geo_cleaned.write.mode("overwrite").parquet(output_path_geo)

# Show the cleaned DataFrame
df_geo_cleaned.show()


+-----+--------------------+--------------------+-------------------+
 ind| country| coordinates| timestamp|
+-----+--------------------+--------------------+-------------------+
 2418|Antarctica (the t...|[-88.4642, -171.061]|2022-05-27 11:30:59|
10794|Cocos (Keeling) I...|[-89.5236, -154.567]|2022-01-01 02:26:50|
 2074|Central African R...| [-52.3213, -50.11]|2019-11-03 05:41:59|
 2293|British Virgin Is...|[-87.7946, -159.647]|2022-03-21 10:46:53|
10663|Saint Kitts and N...| [-27.3474, -162.83]|2019-07-25 18:53:51|
 7922| Antigua and Barbuda|[-88.0974, -172.052]|2021-01-27 09:14:19|
 8606| Antigua and Barbuda|[-88.0974, -172.052]|2021-03-28 14:54:07|
 603|Netherlands Antilles| [14.0083, -141.603]|2019-06-25 05:13:01|
 9979| Dominican Republic| [14.9967, -120.682]|2018-07-18 19:01:46|
 1704| Christmas Island|[-56.8702, -93.6232]|2018-11-29 04:46:39|
 7790| Papua New Guinea| [-43.692, 64.9839]|2018-07-31 08:19:15|
 9759| American Samoa|[-77.9744, -106.258]|2017-12-30 13:05:49|
 1422| American Samoa|[-88.5252, -172.436]|2018-04-30 08:27:21|
 2923| Cote d'Ivoire|[-84.6302, -164.507]|2019-09-08 22:53:09|
 8304| French Guiana| [-28.8852, -164.87]|2019-09-13 04:50:29|
 4315| Cote d'Ivoire| [-45.8508, 66.1003]|2019-12-15 03:51:28|
 8578| Guinea-Bissau| [34.6427, -161.397]|2018-01-26 19:27:11|
 6844| New Caledonia| [-22.6915, 5.69245]|2021-06-06 21:53:11|
 3599| Afghanistan|[-88.5478, -174.971]|2019-03-03 06:13:41|
 5069| Azerbaijan|[-63.0063, -157.474]|2021-03-20 09:32:44|
+-----+--------------------+--------------------+-------------------+
only showing top 20 rows

In [ ]:
#Cleaning the df_user dataframe

from pyspark.sql.functions import col, concat_ws, to_timestamp

# Create a new column 'user_name' that concatenates the information found in the first_name and last_name columns
df_user_cleaned = df_user.withColumn("user_name", concat_ws(" ", col("first_name"), col("last_name")))

# Drop the first_name and last_name columns from the DataFrame
df_user_cleaned = df_user_cleaned.drop("first_name", "last_name")

# Convert the date_joined column from a string to a timestamp data type
df_user_cleaned = df_user_cleaned.withColumn("date_joined", to_timestamp(col("date_joined")))

# Reorder the DataFrame columns
column_order_user = ["ind", "user_name", "age", "date_joined"]
df_user_cleaned = df_user_cleaned.select(column_order_user)

# Get the current timestamp
timestamp_str_user = spark.sql("SELECT current_timestamp() AS timestamp").collect()[0].timestamp

# Create a new output path with a timestamp
output_path_user = f"/mnt/diane_mount/cleaned_data/df_user_cleaned_{timestamp_str_user}"

# Save the cleaned DataFrame to the new output path
df_user_cleaned.write.mode("overwrite").parquet(output_path_user)

# Show the cleaned DataFrame
df_user_cleaned.show()


+-----+--------------------+---+-------------------+
 ind| user_name|age| date_joined|
+-----+--------------------+---+-------------------+
 2015|Christopher Bradshaw| 27|2016-03-08 13:38:37|
 6398| Christina Davenport| 39|2016-06-29 20:43:59|
 3599| Alexandria Alvarado| 20|2015-10-23 04:13:23|
 5076| Christopher Butler| 20|2015-12-01 15:08:31|
 7790| Michelle Gutierrez| 39|2017-07-19 07:12:04|
10509| Brittany Thompson| 49|2016-04-22 20:36:02|
 8731| Andrea Alexander| 21|2015-11-10 09:27:42|
 8887| Austin Rodriguez| 24|2016-03-31 20:56:39|
 7768| Christine Cortez| 23|2015-12-01 18:15:02|
 4315| Michelle Prince| 36|2015-12-20 16:38:13|
10625| Christian Lang| 32|2017-10-10 20:09:33|
 9672| Jennifer Hudson| 22|2016-02-11 20:46:04|
 8930| Andrew Anderson| 23|2015-11-28 11:52:37|
 6566| Alexander Perez| 31|2017-08-04 14:30:22|
 3729| Richard Edwards| 52|2016-02-07 20:00:25|
10552| Michael Hunter| 40|2017-05-16 07:09:21|
 1313| Brittany Jones| 32|2016-04-02 03:51:23|
10794| Thomas Turner| 34|2016-12-22 00:02:02|
 2959| David Griffith| 20|2016-01-07 19:49:22|
 9875| Brendan Joseph| 26|2015-12-20 10:28:00|
+-----+--------------------+---+-------------------+
only showing top 20 rows

##Data Transform & Query the DataFrames

In [ ]:
#Find the most Popular Catergory in Each Country

from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime

# Get current timestamp
current_timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# Join df_pin_cleaned and df_geo_cleaned on 'ind'
joined_df_pin_geo = df_pin_cleaned.join(df_geo_cleaned, 'ind')

# Join the combined dataframe with df_user_cleaned on 'ind'
joined_df = joined_df_pin_geo.join(df_user_cleaned, 'ind')

# Group by 'country' and 'category' and count occurrences
category_counts = joined_df.groupBy('country', 'category').count()

# Rank categories within each country based on count
window_spec = Window.partitionBy('country').orderBy(F.desc('count'))
ranked_df = category_counts.withColumn('category_rank', F.rank().over(window_spec))

# Filter for the top-ranked category in each country
most_popular_category = ranked_df.filter('category_rank == 1').drop('category_rank', 'count')

# Save the DataFrame to the specified mount point with a timestamp
output_path_popular_category = f"/mnt/diane_mount/transformed_data/most_popular_category_{current_timestamp}"
most_popular_category.write.mode("overwrite").parquet(output_path_popular_category)

# Show the DataFrame
most_popular_category.show()


+--------------------+--------------+
 country| category|
+--------------------+--------------+
 Afghanistan| education|
 Afghanistan| art|
 Albania| mens-fashion|
 Algeria| quotes|
 Algeria| beauty|
 Algeria| travel|
 American Samoa| travel|
 American Samoa| beauty|
 Andorra| art|
 Angola|diy-and-crafts|
 Anguilla| home-decor|
 Anguilla| beauty|
Antarctica (the t...| christmas|
 Antigua and Barbuda| tattoos|
 Antigua and Barbuda| quotes|
 Argentina| beauty|
 Argentina| tattoos|
 Armenia|diy-and-crafts|
 Aruba| tattoos|
 Australia| mens-fashion|
+--------------------+--------------+
only showing top 20 rows

In [ ]:
#Find Which was The Most Popular Category Each Year between 2018 and 2022

from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime

# Join df_pin_cleaned and df_geo_cleaned on 'ind'
joined_df = df_pin_cleaned.join(df_geo_cleaned, 'ind')

# Extract the year from the timestamp column
df_pin_year = joined_df.withColumn("post_year", F.year("timestamp"))

# Filter for posts between 2018 and 2022
df_pin_filtered = df_pin_year.filter((F.col("post_year") >= 2018) & (F.col("post_year") <= 2022))

# Group by 'post_year', 'category' and count occurrences
category_counts_by_year = df_pin_filtered.groupBy("post_year", "category").count()

# Rank categories within each year based on count
window_spec = Window.partitionBy('post_year').orderBy(F.desc('count'))
ranked_df = category_counts_by_year.withColumn('category_rank', F.rank().over(window_spec))

# Filter for the top-ranked category in each year
most_popular_category_by_year = ranked_df.filter('category_rank == 1').drop('category_rank', 'count')

# Get current timestamp
current_timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# Save the DataFrame to the specified mount point with timestamp
output_path_category_counts_by_year = f"/mnt/diane_mount/transformed_data/category_counts_by_year_{current_timestamp}"
most_popular_category_by_year.write.mode("overwrite").parquet(output_path_category_counts_by_year)

# Show the DataFrame
most_popular_category_by_year.show()




+---------+--------------+
post_year| category|
+---------+--------------+
 2018| beauty|
 2019| education|
 2020| art|
 2021|diy-and-crafts|
 2021| finance|
 2021| tattoos|
 2022| christmas|
 2022| beauty|
+---------+--------------+

In [ ]:
#find the user with the most followers in each country

from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Assuming 'ind' is a common column between df_pin_cleaned and df_geo_cleaned
joined_df = df_pin_cleaned.join(df_geo_cleaned.select("ind", "country"), "ind")

# Now you can use the 'country' column from df_geo_cleaned in df_pin_cleaned
df_pin_cleaned_with_country = joined_df.select("ind", "unique_id", "title", "description", "follower_count",
                                              "poster_name", "tag_list", "is_image_or_video", "image_src",
                                              "save_location", "category", "country")

# Rank users within each country based on follower count
window_spec = Window.partitionBy('country').orderBy(F.desc('follower_count'))
ranked_df = df_pin_cleaned_with_country.withColumn('user_rank', F.rank().over(window_spec))

# Get the user with the most followers in each country
top_user_per_country = ranked_df.filter('user_rank == 1').drop('user_rank')

# Save the DataFrame to the specified mount point with timestamp
output_path_top_user_per_country = f"/mnt/diane_mount/transformed_data/top_user_per_country_{current_timestamp}"
top_user_per_country.write.mode("overwrite").parquet(output_path_top_user_per_country)

# Show the DataFrame
top_user_per_country.show()

# Now, find the country with the user with the most followers
most_followed_country = top_user_per_country.select("country", "follower_count").orderBy(F.desc("follower_count")).limit(1)

# Save the DataFrame to the specified mount point with timestamp
output_path_most_followed_country = f"/mnt/diane_mount/transformed_data/most_followed_country_{current_timestamp}"
most_followed_country.write.mode("overwrite").parquet(output_path_most_followed_country)

# Show the DataFrame
most_followed_country.show()


+------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+--------------------+
 ind| unique_id| title| description|follower_count| poster_name| tag_list| is_image_or_video| image_src| save_location| category| country|
+------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+--------------------+
3599.0|ff0dd945-dafa-411...|Educational Leade...|This book present...| null| Walmart|Research Studies,...| image|https://i.pinimg....| education| education| Afghanistan|
 46.0|19234073-8905-488...|10 Watercolor Hac...|Mountain monologu...| null|It's me, JD | DIY...|Arte Inspo,Kunst ...| image|https://i.pinimg....| art| art| Afghanistan|
2411.0|4e11801e-7a64-4f1...|The Sweetest Ging...|Transform mini te...| null| Projects with Kids|Noel Christmas,Di...| image|https://i.pinimg....| christmas| christmas| Albania|
6398.0|ea44d318-2ced-4cf...|My master bathroo...|No description av...| null| MojisStyle|Decor Home Living...|multi-video(story...|https://i.pinimg....| home-decor| home-decor| Albania|
 808.0|23f67e42-db9f-4aa...|Chalk Pastel Hot ...|Kids will use cha...| null| Projects with Kids|Hot Chocolate Art...| image|https://i.pinimg....| art| art| Albania|
7528.0|fbe53c66-3442-477...|No Title Data Ava...|No description av...| null| User Info Error|N,o, ,T,a,g,s, ,A...|multi-video(story...| Image src error.| mens-fashion| mens-fashion| Albania|
7528.0|fbe53c66-3442-477...|No Title Data Ava...|No description av...| null| User Info Error|N,o, ,T,a,g,s, ,A...|multi-video(story...| Image src error.| mens-fashion| mens-fashion| Albania|
3089.0|88f9227e-88d0-4b1...|No Title Data Ava...|No description av...| null| User Info Error|N,o, ,T,a,g,s, ,A...|multi-video(story...| Image src error.|diy-and-crafts|diy-and-crafts| Albania|
1555.0|1f20551b-a21c-4ff...|Skin Purging: Die...|Skin Purging sorg...| null| ELLE Germany|Natural Hair Mask...| image|https://i.pinimg....| beauty| beauty| Algeria|
7768.0|ae1a61f6-d958-40f...|16 Motivational Q...| motivation quotes| null|Boss Babe Chronic...|Motivacional Quot...| image|https://i.pinimg....| quotes| quotes| Algeria|
9590.0|b849e976-52f6-45d...|Top 20 Things To ...|Best spots to see...| null| Fun Life Crisis|Vacation Ideas,Va...| image|https://i.pinimg....| travel| travel| Algeria|
9759.0|d105eb6e-0f9f-46e...|8 Best Greek Isla...|Although you'd th...| null|TheFab20s | Trave...|Greek Islands To ...| image|https://i.pinimg....| travel| travel| American Samoa|
1422.0|7bbe860c-a739-4ba...|41 Life-Saving Be...|Might try this......| null| BuzzFeed|Baking Soda Lemon...| image|https://i.pinimg....| beauty| beauty| American Samoa|
 159.0|841a161a-47b8-416...|Valentine's Day B...|This bee directed...| null|Teachers Pay Teac...|Classroom Art Pro...| image|https://i.pinimg....| art| art| Andorra|
2959.0|439cac4b-18e8-4aa...|The Cutest Toilet...|One thing I love ...| null| CraftGossip.com|Toilet Paper Roll...| image|https://i.pinimg....|diy-and-crafts|diy-and-crafts| Angola|
6063.0|60693727-4927-4bd...|41 Gorgeous Fall ...|Beautiful and eas...| null|Kristen | Lifesty...|Fall Home Decor,A...| image|https://i.pinimg....| home-decor| home-decor| Anguilla|
1545.0|0c629541-cc5b-4b6...|Oil Cleansing Met...|The oil cleansing...| null| Thank Your Body|Beauty Care,Diy B...| image|https://i.pinimg....| beauty| beauty| Anguilla|
2418.0|da8745a6-5160-46c...|100 DIY Christmas...|Here are the best...| null| HikenDip|Farmhouse Christm...| image|https://i.pinimg....| christmas| christmas|Antarctica (the t...|
8606.0|71757040-bfc8-463...|30+ Eagle Tattoos...|summcoco gives yo...| null|Sumcoco | Decor I...|White Bird Tattoo...| image|https://i.pinimg....| tattoos| tattoos| Antigua and Barbuda|
7922.0|a584581c-1b38-473...|45 Top Life Quote...|summcoco gives 

In [ ]:
#Task 7 Find the Most Popular Category for Different Age Groups

from pyspark.sql import functions as F

# Extract age group from the age column
df_user_age_group = df_user_cleaned.withColumn("age_group", F.when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                               .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                               .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                               .when(col("age") > 50, "+50")
                                               .otherwise("Unknown"))

# Join df_pin_cleaned and df_user_age_group on 'ind'
joined_df_age_group = df_pin_cleaned.join(df_user_age_group, 'ind')

# Group by 'age_group' and 'category' and count occurrences
category_counts_by_age_group = joined_df_age_group.groupBy('age_group', 'category').count()

# Rank categories within each age group based on count
window_spec_age_group = Window.partitionBy('age_group').orderBy(F.desc('count'))
ranked_df_age_group = category_counts_by_age_group.withColumn('category_rank', F.rank().over(window_spec_age_group))

# Filter for the top-ranked category in each age group
most_popular_category_by_age_group = ranked_df_age_group.filter('category_rank == 1').drop('category_rank', 'count')

# Get current timestamp
current_timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# Save the DataFrame to the specified mount point
output_path_category_counts_by_age_group = f"/mnt/diane_mount/transformed_data/category_counts_by_age_group_{current_timestamp}"
most_popular_category_by_age_group.write.mode("overwrite").parquet(output_path_category_counts_by_age_group)

# Show the DataFrame
most_popular_category_by_age_group.show()


+---------+--------------+
age_group| category|
+---------+--------------+
 +50| education|
 +50| beauty|
 18-24| tattoos|
 18-24| mens-fashion|
 18-24| christmas|
 25-35| art|
 25-35| travel|
 36-50|diy-and-crafts|
+---------+--------------+

In [ ]:
#Find the Median Follower Count for Different Age Groups

from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Extract age group from the age column in df_user
df_user_age_group = df_user_cleaned.withColumn("age_group", F.when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                               .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                               .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                               .when(col("age") > 50, "+50")
                                               .otherwise("Unknown"))

# Join df_pin_cleaned and df_user_age_group on 'ind'
joined_df_age_group_follower_count = df_pin_cleaned.join(df_user_age_group, 'ind')

# Group by 'age_group' and calculate median follower count
median_follower_count_by_age_group = joined_df_age_group_follower_count.groupBy('age_group') \
    .agg(F.expr('percentile(follower_count, 0.5)').alias('median_follower_count'))

# Get current timestamp
current_timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# Save the DataFrame to the specified mount point
output_path_median_follower_count_by_age_group = f"/mnt/diane_mount/transformed_data/median_follower_count_by_age_group_{current_timestamp}"
median_follower_count_by_age_group.write.mode("overwrite").parquet(output_path_median_follower_count_by_age_group)

# Show the DataFrame
median_follower_count_by_age_group.show()


+---------+---------------------+
age_group|median_follower_count|
+---------+---------------------+
 36-50| 0.0|
 +50| 55.5|
 18-24| 313.0|
 25-35| 407.0|
+---------+---------------------+

In [ ]:
#Find out How Many Users Have Joined Each Year Between 2015 and 2020

from pyspark.sql.functions import year

# Extract the year from the date_joined column in df_user_cleaned
df_user_joined_year = df_user_cleaned.withColumn("join_year", year("date_joined"))

# Filter for users who joined between 2015 and 2020
df_user_filtered_joined_year = df_user_joined_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Group by 'join_year' and count the number of users
number_users_joined_by_year = df_user_filtered_joined_year.groupBy("join_year") \
    .agg(F.count("ind").alias("number_users_joined"))

# Get current timestamp
current_timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# Save the DataFrame to the specified mount point
output_path_number_users_joined_by_year = f"/mnt/diane_mount/transformed_data/number_users_joined_by_year_{current_timestamp}"
number_users_joined_by_year.write.mode("overwrite").parquet(output_path_number_users_joined_by_year)

# Show the DataFrame
number_users_joined_by_year.show()




+---------+-------------------+
join_year|number_users_joined|
+---------+-------------------+
 2015| 33|
 2016| 36|
 2017| 11|
+---------+-------------------+

In [ ]:
#Find the median follower count of users have joined between 2015 and 2020.

from pyspark.sql.functions import year, expr

# Join df_user_cleaned and df_pin_cleaned on 'ind'
joined_df_user_pin = df_user_cleaned.join(df_pin_cleaned, 'ind')

# Extract the year from the date_joined column in df_user_cleaned
df_user_joined_year = joined_df_user_pin.withColumn("join_year", year("date_joined"))

# Filter for users who joined between 2015 and 2020
df_user_filtered_joined_year = df_user_joined_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Group by 'join_year' and calculate the median follower count
median_follower_count_by_year = df_user_filtered_joined_year.groupBy("join_year") \
    .agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

# Get current timestamp
current_timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# Save the DataFrame to the specified mount point
output_path_median_follower_count_by_year = f"/mnt/diane_mount/transformed_data/median_follower_count_by_year_{current_timestamp}"
median_follower_count_by_year.write.mode("overwrite").parquet(output_path_median_follower_count_by_year)

# Show the DataFrame
median_follower_count_by_year.show()


+---------+---------------------+
join_year|median_follower_count|
+---------+---------------------+
 2015| 0|
 2016| 111|
 2017| 112|
+---------+---------------------+

In [ ]:
# Task 11: Find the Median Follower Count of Users based on their joining year and age group

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, expr, percentile_approx, year

# Get current timestamp
current_timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# Filter for users who joined between 2015 and 2020
df_user_filtered_joined_year = df_user_cleaned.filter((col("date_joined").between('2015-01-01', '2020-12-31')))

# Define age groups and corresponding labels
age_groups = [
    (col("age") >= 18) & (col("age") <= 24),
    (col("age") >= 25) & (col("age") <= 35),
    (col("age") >= 36) & (col("age") <= 50),
    (col("age") > 50)
]

age_group_labels = ["18-24", "25-35", "36-50", "+50"]

# Use a when statement to categorize users into age groups
for age_condition, label in zip(age_groups, age_group_labels):
    df_user_filtered_joined_year = df_user_filtered_joined_year \
        .withColumn("age_group", when(age_condition, label))

# Extract the year from the timestamp column in df_user_cleaned
df_user_filtered_joined_year = df_user_filtered_joined_year \
    .withColumn("post_year", year("date_joined"))

# Calculate median follower count for each age group and post_year
median_follower_count_by_age_and_year = df_user_filtered_joined_year \
    .join(df_pin_cleaned, 'ind') \
    .groupBy("age_group", "post_year") \
    .agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

# Drop null values if any
median_follower_count_by_age_and_year = median_follower_count_by_age_and_year.na.drop()

# Save the DataFrame to the specified mount point
output_path_median_follower_count_by_age_and_year = f"/mnt/diane_mount/transformed_data/median_follower_count_by_age_and_year_{current_timestamp}"
median_follower_count_by_age_and_year.write.mode("overwrite").parquet(output_path_median_follower_count_by_age_and_year)

# Show the DataFrame
median_follower_count_by_age_and_year.show()



+---------+---------+---------------------+
age_group|post_year|median_follower_count|
+---------+---------+---------------------+
 +50| 2016| 0|
+---------+---------+---------------------+